<a href="https://colab.research.google.com/github/emoceanographer/usdanutrients/blob/master/Protein_requirements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import re
import statistics
from operator import add
import numpy as np

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Nutrition/'
pop_path = path + 'WPP2019_PopulationByAgeSex_Medium.csv'
BMI_path = path + 'NCD_BMI_MEAN.csv'
height_path = path + 'heights_over_time.csv'
fao_ctry_path = path + 'fao_ctry_codes.csv'

# Calculate avg. body mass of adults for each country

Population: Dataset is WPP2019, Population by age and sex, medium; for each country; plan is to, for each (country, year), match demographic groups to the age_aa requirement pattern and calculate needed aa requirements

In [0]:
pop_df = pd.read_csv(pop_path, encoding='latin-1') # each country has year, age group's population
BMI_df = pd.read_csv(BMI_path) # each country has a year and an associated mean
height_df = pd.read_csv(height_path)
fao_ctry_codes = pd.read_csv(fao_ctry_path)

In [0]:
BMI_df = BMI_df.rename(columns ={'Unnamed: 0':'Country'})

In [0]:
# Age group definitions
child_age_grps = ['0-4', '5-9', '10-14', '15-19']
adult_age_grps = ['20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74',
       '75-79', '80-84', '85-89', '90-94', '95-99', '100+'] # age groups for pop. data

In [0]:
ctry_mismatches = {'China, Hong Kong SAR': ['none'], 'China, Macao SAR': ['none'], 
                   'China, Taiwan Province of China': ['none'], 
                   "Dem. People's Republic of Korea": "Democratic People's Republic of Korea",
                   'French Guiana': ['none'], 'French Polynesia': ['none'], 
                   'Guadeloupe': ['none'],'Martinique': ['none'],
                   'United Kingdom': ['United Kingdom of Great Britain and Northern Ireland']}

In [0]:
# for each year and country, calculate the average mass of an adult
avg_weight = {}
for country in list(pop_df.Location.unique()): # cycle through each country
  # population
  temp = pop_df[pop_df['Location'].isin([country])]
  temp = temp[temp['AgeGrp'].isin(adult_age_grps)]
  
  # BMI
  ctr = 0
  if country in BMI_df['Unnamed: 0'].unique(): # skip countries for which there is no BMI
    ctr = 1
    BMI = BMI_df[BMI_df['Unnamed: 0'] == country]
    
  if country in ctry_mismatches: 
    if ctry_mismatches[country][0] in BMI_df['Unnamed: 0'].unique():
      ctr = 1
      BMI = BMI_df[BMI_df['Unnamed: 0'] == ctry_mismatches[country][0]]
  
  if ctr == 1:

    if country == 'United Kingdom':
      ctry_code = 'GBR'
    else:
      ctry_code = fao_ctry_codes[fao_ctry_codes['Short name'] == country]['ISO3'].values[0]
    
    # Heights (gets people 20-40 in 2011-2013 by year of birth)
    hts = height_df[height_df['Code'] == ctry_code]
    hts_m = hts[(hts['Year']>1970) & (hts['Year']<1994)]['Males (centimeters)'].mean()
    hts_f = hts[(hts['Year']>1970) & (hts['Year']<1994)]['Females (centimeters)'].mean()
    
    f_pop_year = []
    m_pop_year = []
    f_BMI = []
    m_BMI = []
    for year in list(pop_df.Time.unique()):
      if year < 2014 and year > 2010: # get 2011-2013
        # population
        f_adult_yr_pop = sum(temp[temp['Time'].isin([year])]['PopFemale'])
        m_adult_yr_pop = sum(temp[temp['Time'].isin([year])]['PopMale'])
        f_pop_year.append(f_adult_yr_pop*1000) # units originally in thousands of people
        m_pop_year.append(m_adult_yr_pop*1000)

        # BMI (average only)
        yr_str = str(year)
        BMI_f_temp = re.search(r'^.*(?=(\[))', BMI[yr_str + '.2'].values[0])
        BMI_m_temp = re.search(r'^.*(?=(\[))', BMI[yr_str + '.1'].values[0])
        if BMI_f_temp:
          f_BMI.append(float(BMI_f_temp.group(0)))
        else:
          f_BMI.append(np.nan)
        if BMI_m_temp:
          m_BMI.append(float(BMI_m_temp.group(0)))
        else:
          m_BMI.append(np.nan)
    
    # Weight calculation (kg)
    f_weight = (hts_f/100)**2 * statistics.mean(f_BMI)
    m_weight = (hts_m/100)**2 * statistics.mean(m_BMI)
    
    # Population weighting
    avg_weight[country] = (f_weight*statistics.mean(f_pop_year) + m_weight*statistics.mean(m_pop_year)) / statistics.mean(list(map(add, f_pop_year, m_pop_year)))
  else:
    print(country)

Africa
Aruba
Asia
Australia/New Zealand
Caribbean
Central America
Central and Southern Asia
Central Asia
Channel Islands
China, Hong Kong SAR
China, Macao SAR
China, Taiwan Province of China
Curaçao
Dem. People's Republic of Korea
Eastern Africa
Eastern and South-Eastern Asia
Eastern Asia
Eastern Europe
Europe
Europe and Northern America
French Guiana
French Polynesia
Guadeloupe
Guam
High-income countries
Land-locked Developing Countries (LLDC)
Latin America and the Caribbean
Least developed countries
Less developed regions
Less developed regions, excluding China
Less developed regions, excluding least developed countries
Low-income countries
Lower-middle-income countries
Martinique
Mayotte
Melanesia
Micronesia
Micronesia (Fed. States of)
Middle Africa
Middle-income countries
More developed regions
New Caledonia
No income group available
North Macedonia
Northern Africa
Northern Africa and Western Asia
Northern America
Northern Europe
Oceania
Oceania (excluding Australia and New Zealand

In [0]:
ctry_weights = pd.DataFrame.from_dict(avg_weight,orient='index')
ctry_weights = ctry_weights.reset_index()
missing = ctry_weights[ctry_weights[0].isna()]
list(missing['index'].unique()) # should be just those with actually no data (e.g. Sudan)



['South Sudan', 'Sudan']

In [0]:
ctry_weights

,index,0
0,Afghanistan,59.128907
1,Albania,74.300504
2,Algeria,69.186634
3,Angola,61.446617
4,Antigua and Barbuda,72.731290
...,...,...
175,Venezuela (Bolivarian Republic of),71.597418
176,Viet Nam,53.435563
177,Yemen,59.061992
178,Zambia,59.035999


# Calculate AA requirements per capita (adults)

In [0]:
case = 'baseline'

if case == 'baseline':

  AA_adult_req = {'Histidine': 15, 'Isoleucine': 30, 'Leucine': 59, 'Lysine': 45, 
                 'SAA': 22, 'AAA': 38, 'Threonine': 23, 'Tryptophan': 6, 'Valine': 39} # mg AA /g protein
  AA_total = .66 # g/kg
elif case == 'safe':
  AA_adult_req = {'Histidine': 12, 'Isoleucine': 24, 'Leucine': 47, 'Lysine': 36, 
                 'SAA': 17, 'AAA': 30, 'Threonine': 18, 'Tryptophan': 5, 'Valine': 31} # mg AA /g protein
  AA_total = .83 # g/kg

In [0]:
SAA = ['Methionine', 'Cystine'] # there are others, but not in my diet AA list
AAA = ['Phenylalanine', 'Tyrosine'] # also tryptophan and histidine but they are listed separately

In [0]:
diet_AA = pd.read_csv(path + 'diet_aa_1030.csv')

In [0]:
AA_ctry = {}
AA_all = {}
for country in diet_AA['Unnamed: 0'].unique().tolist():
  if country in ctry_weights['index'].unique().tolist():
    diet_ctry = diet_AA[diet_AA['Unnamed: 0'] == country]
    AA_ctry[country] = {}
    AA_all[country] = {}
    for AA in AA_adult_req: 
      avg_weight = ctry_weights[ctry_weights['index']==country][0].values[0] 
      AA_req =  avg_weight * AA_total * AA_adult_req[AA] * 0.001 #g/capita/day
      if AA == 'SAA':
        aa_diet = []
        for aa in SAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      elif AA == 'AAA':
        aa_diet = []
        for aa in AAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      else:
        AA_diet = diet_ctry[AA].values[0]
      AA_ctry[country][AA] = AA_diet / AA_req #[AA_req, AA_diet, AA_diet/AA_req] 
      AA_all[country][AA] = [AA_req, AA_diet, AA_diet/AA_req] 
      if AA_ctry[country][AA]<1:
        print(country, ' lacking ' , AA, ' see: ', AA_req, AA_diet)
    total_protein_req = avg_weight * AA_total
    total_protein_diet = diet_ctry['protein'].values[0]
    AA_ctry[country]['protein'] =  total_protein_diet/total_protein_req #[total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    AA_all[country]['protein'] = [total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    if AA_ctry[country]['protein'] < 1: 
      print(country, ' lacking total protein, see' , total_protein_req, total_protein_diet)

Botswana  lacking total protein, see 44.571305692524426 39.37955272541638
Burkina Faso  lacking total protein, see 39.94765649450399 37.68984071169531
Cameroon  lacking  Leucine  see:  2.5610903530351123 2.505075051222612
Cameroon  lacking  Lysine  see:  1.9533739980776283 1.9024724893831808
Cameroon  lacking  Valine  see:  1.6929241316672778 1.4988342785909057
Cameroon  lacking total protein, see 43.40831106839174 32.771679563383344
Central African Republic  lacking total protein, see 39.838969339630815 38.88468435925096
Chad  lacking  Isoleucine  see:  1.2040984671960073 1.0413191745254369
Chad  lacking  Leucine  see:  2.3680603188188143 2.1126089379569803
Chad  lacking  Lysine  see:  1.8061477007940112 1.551967972852235
Chad  lacking  SAA  see:  0.8830055426104054 0.8716101965293748
Chad  lacking  Threonine  see:  0.9231421581836057 0.8896969291788854
Chad  lacking  Valine  see:  1.5653280073548097 1.2024736149059008
Chad  lacking total protein, see 40.136615573200245 28.16419555456

In [0]:
AA_ratios = pd.DataFrame.from_dict(AA_ctry,orient='index')

In [0]:
savepath = path + 'AA_ratios_safe.csv'
AA_ratios.to_csv(savepath)

In [0]:
AA_all['Zimbabwe']

{'AAA': [1.6082451032370018, 2.624873961736261, 1.632135522410754],
 'Histidine': [0.634833593383027, 0.8999105221507897, 1.4175534053816659],
 'Isoleucine': [1.269667186766054, 1.4086851410921415, 1.109491649288171],
 'Leucine': [2.4970121339732394, 2.9192874185230693, 1.1691122276918642],
 'Lysine': [1.904500780149081, 2.079767917758042, 1.092027863383307],
 'SAA': [0.9310892702951064, 1.199306323266803, 1.28806803120681],
 'Threonine': [0.9734115098539747, 1.2110487765840383, 1.2441282687994026],
 'Tryptophan': [0.25393343735321083, 0.3070967028506071, 1.209359059017692],
 'Valine': [1.6505673427958703, 1.6551851319766195, 1.0027976981374944],
 'protein': [42.32223955886847, 34.83671037578432, 0.8231301258840027]}

## Undernourishment
Use the average diet as the mean of lognormal & the avg. requirement as threshold for undernourishment (NB FAO would use a 5th percentile for this?)

In [0]:
import math
import scipy
from scipy import stats
from scipy.optimize import least_squares

pr_mean = 50 # sample
sigma = 0.25 # sample

req = 40 # sample

mu = math.log(pr_mean) - sigma**2/2
CDF = 1/2 + 1/2*scipy.special.erf((math.log(req) - mu)/(2**0.5*sigma))


In [11]:
CDF

0.22137009599489793

# Old Code

In [0]:
# BMI regex work! (save me for now!)
temp = BMI['2016'].values[0]
print(temp)
m = re.search(r"[^[]*\[([^]]*)\]", temp).groups()[0] # gets the things in the brackets!
n = re.search(r'^.*(?=(\[))', temp).group(0)
print(m)
print(n)

23.4 [22.0-24.8]
22.0-24.8
23.4 


In [0]:
adult_weight = 136.68 # average adult weight


In [0]:
# calculate average height (m) for each country; look into this later
ctry_hts = {}
for country in list(pop_df.Location.unique()): 
  if country in list(height_df.Entity.unique()):
    ht = height_df[height_df['Entity'].isin([country])]
    m_ht = statistics.mean(ht['Males (centimeters)'])
    f_ht = statistics.mean(ht['Females (centimeters)'])
    ctry_hts[country] = {'M': m_ht/100, 'F': f_ht/100}

In [0]:
# match those historical populations with average weight
avg_adult_weight = {}
total_adult_weight = {}

for country in list(pop_df.Location.unique()): 
  BMI = BMI_df[BMI_df['Country'].isin([country])]
  f_yrs_weight = {}
  m_yrs_weight = {}
  
  ctry_ht = ctry_hts.get(country)
  if ctry_ht:
    m_ht = ctry_ht['M'] # get the average height for that country
    f_ht = ctry_ht['F']

    f_tot_weight = {} # pre-allocate
    m_tot_weight = {}
    tot_yr_weight = {}
    for year in list(pop_df.Time.unique()):
      # BMI
      if str(year) in BMI.columns: # only go through years for which we have data
        avg_m_BMI = re.sub(r'\[[^]]*\]','',BMI[str(year)+'.1'].item())
        avg_f_BMI = re.sub(r'\[[^]]*\]','',BMI[str(year)+'.2'].item())
        
        f_yrs_weight[year] = float(avg_f_BMI)*(f_ht)**2 # calculates weight from BMI and height
        m_yrs_weight[year] = float(avg_m_BMI)*(m_ht)**2
        
        f_tot_weight[year] = f_adult_population[country][year]*f_yrs_weight[year]
        m_tot_weight[year] = m_adult_population[country][year]*m_yrs_weight[year]
        tot_yr_weight[year] = f_tot_weight[year] + m_tot_weight[year]
        
    avg_adult_weight[country] = {'f': f_yrs_weight, 'm': m_yrs_weight}
  
    total_adult_weight[country] = tot_yr_weight
  else:
    print([country, ' did not have associated height'])

['Africa', ' did not have associated height']
['Aruba', ' did not have associated height']
['Asia', ' did not have associated height']
['Australia/New Zealand', ' did not have associated height']
['Bolivia (Plurinational State of)', ' did not have associated height']
['Brunei Darussalam', ' did not have associated height']
['Cabo Verde', ' did not have associated height']
['Caribbean', ' did not have associated height']
['Central America', ' did not have associated height']
['Central and Southern Asia', ' did not have associated height']
['Central Asia', ' did not have associated height']
['Channel Islands', ' did not have associated height']
['China, Hong Kong SAR', ' did not have associated height']
['China, Macao SAR', ' did not have associated height']
['China, Taiwan Province of China', ' did not have associated height']
["Côte d'Ivoire", ' did not have associated height']
['Curaçao', ' did not have associated height']
['Czechia', ' did not have associated height']
["Dem. People's

ValueError: ignored